In [1]:
import numpy as np
import re
import random

import tensorflow as tf
import sklearn as sk

import Functions as fn
from DS import DS
from Set import pool
from Iterator import Iterator

In [2]:
#Dataset = fn.first_time_load()
#Dataset.write_texts()
#Dataset.write_labels()

In [3]:
Dataset = pool()
Dataset.load_texts()
Dataset.load_labels()

Texts loaded
Labels loaded


In [4]:
#print('Number of Texts: ', Dataset.size)
#print('Number of 2007 Smoking Challenge texts: ', Dataset.number_of(challenge='2007 Smoking Challenge'))
#print('Number of 2008 ObesityChallenge texts: ', Dataset.number_of(challenge='2008 Obesity Challenge'))
#print('Number of 2009 Medication Challenge texts: ', Dataset.number_of(challenge='2009 Medication Challenge'))
#print('Number of 2010 Relations Challenge texts: ', Dataset.number_of(challenge='2010 Relations Challenge'))
#print('Number of 2011 Coreference Challenge texts: ', Dataset.number_of(challenge='2011 Coreference Challenge'))
#print('Number of 2012 Temporal Relations Challenge texts: ', Dataset.number_of(challenge='2012 Temporal Relations Challenge'))
#print('Number of Train Texts: ', Dataset.number_of(stage='train'))
#print('Number of Test Texts: ', Dataset.number_of(stage='test'))
#print('Number of Labeled Texts: ', Dataset.number_of(labelled='yes'))
#print('Number of Initially Labeled Texts: ', Dataset.number_of(labelled='yes', label_type='train'))
#print('Number of Competitor Labeled Texts Texts: ', Dataset.number_of(labelled='yes', label_type='test'))

In [5]:
Dataset.process_for_embedding()
sentences = Dataset.get_sentences()

In [6]:
from gensim.models import Word2Vec

C:\Users\Luka\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [7]:
model = Word2Vec.load('W2V')

In [8]:
#model = Word2Vec(sentences, min_count=1, size=100)

In [9]:
#model.most_similar("prozac", topn=10)

In [10]:
targets = ['medications', 'dosages', 'modes', 'frequencies', 'durations', 'reasons']
target_dict = {label: words for (label, words) in zip(targets, list(fn.label_words(Dataset)))}

Number of: m=928, do=119, mo=76, f=79, du=80, r=482


In [11]:
#fn.visualise(model, sentences, [target_dict['medications']], 1000, 'Medications in top 1000 words')

In [12]:
#print_set = set()
#for case in Dataset.get_DS(labelled='yes').data:
#    for term in re.finditer(r'm="[a-z0-9 ]+"', case.raw_labels):
#        if term.group()[3:-1] != 'nm': print_set.add(term.group()[3:-1])
#print(*print_set, sep='\n')

In [13]:
#for target in targets:
#    for reps in [True, False]:
#        target_size = len(target_dict[target])
#        sets = fn.get_traintest(model, target_dict[target], target_size * 5, target_size, 10, 50, reps, reps)
#        fn.model_1(sets[0], sets[1], sets[2], sets[3], target=target, repetitions='yes' if reps else 'no')

In [14]:
#fn.write_emb('testemb', model)

In [15]:
#test_model = fn.load_emb('testemb')

In [16]:
#model.save('W2V')

In [62]:
labelled_cases = Dataset.get_DS(labelled='yes')

In [63]:
len(labelled_cases.data)

258

In [64]:
for case in labelled_cases.data:
    text = case.raw_text
    text = text.split('\n')
    for i in range(len(text)):
        text[i] = text[i].strip('.')
        text[i] = re.sub(r'\d+', '<NUM>', text[i])
        text[i] = re.sub(r'([A-Za-z]):', r'\1', text[i])
        text[i] = re.sub(r'Dr.', 'Dr', text[i])
        text[i] = re.sub(r'Mr.', 'Mr', text[i])
        text[i] = re.sub(r'\. ([A-Z])', r'. A\1', text[i])
        text[i] = re.sub(r'\. [A-Z]', ' ', text[i])
        text[i] = text[i].lower()
        text[i] = text[i].split()
    case.token_text = text
    
    indeces = []
    for med in re.finditer(r'm="[^"]*" \d+:\d+ \d+:\d+', case.raw_labels):
        indeces.append([[int(a) for a in b.split(':')] for b in med.group().split()[-2:]])
    indeces.sort()
    indeces.append([[0,0],[0,0]])
    
    truth = []
    c = 0
    inside =  False
    for i in range(len(text)):
        for j in range(len(text[i])):
            if inside:
                if i+1 < indeces[c][1][0]: truth.append([1,0])
                elif i+1 == indeces[c][1][0]:
                    if j < indeces[c][1][1]: truth.append([1,0])
                    elif j == indeces[c][1][1]:
                        truth.append([1,0])
                        inside = False
                        c += 1
            else:
                if [i+1, j] == indeces[c][0]:
                    truth.append([1,0])
                    if [i+1, j] == indeces[c][1]: c += 1
                    else: inside = True
                else: truth.append([0,1])
    case.test_labels = truth
    
    case.test_text = [word for sentence in case.token_text for word in sentence]

In [65]:
for i in range(len(labelled_cases.data)):
    if len(labelled_cases.data[i].test_labels) != len(labelled_cases.data[i].test_text):
        print(i, len(labelled_cases.data[i].test_labels), len(labelled_cases.data[i].test_text))

240 295 548


In [61]:
indeces = []
for med in re.finditer(r'm="[^"]*" \d+:\d+ \d+:\d+', labelled_cases.data[240].raw_labels):
    indeces.append([[int(a) for a in b.split(':')] for b in med.group().split()[-2:]])
    print(med.group())
indeces.sort()
indeces.append([[0,0],[0,0]])
print(indeces)
print(len(indeces))

m="lantus ( insulin glargine )" 17:0 17:4
m="ranitidine hcl syrup" 19:0 19:2
m="roxicet elixir ( oxycodone+apap liquid )" 20:0 20:5
m="colace ( docusate sodium )" 22:0 22:4
m="phenergan ( promethazine hcl )" 24:0 24:4
m="augmentin susp. 250mg/62.5 mg ( 5ml ) ( amoxicil... )" 25:0 25:9
m="pca analgesia" 51:11 51:12
m="elixir analgesia" 52:7 52:8
m="roxicet" 59:13 59:13
m="qugmentin suspension" 60:9 60:10
m="ranitidine elixir" 60:1 60:2
m="previous home medications." 61:1 61:3
m="lantus" 63:4 63:4
m="narcotic ( pain ) medications." 70:5 70:9
[[[17, 0], [17, 4]], [[19, 0], [19, 2]], [[20, 0], [20, 5]], [[22, 0], [22, 4]], [[24, 0], [24, 4]], [[25, 0], [25, 9]], [[51, 11], [51, 12]], [[52, 7], [52, 8]], [[59, 13], [59, 13]], [[60, 1], [60, 2]], [[60, 9], [60, 10]], [[61, 1], [61, 3]], [[63, 4], [63, 4]], [[70, 5], [70, 9]], [[0, 0], [0, 0]]]
15


In [59]:
print(labelled_cases.data[240].raw_labels)
print(len(labelled_cases.data[240].raw_labels.split('\n')))
labelled_cases.data[240].show_info()

m="lantus ( insulin glargine )" 17:0 17:4||do="10 units" 17:5 17:6||mo="sc" 17:7 17:7||f="qd" 17:8 17:8||du="nm"||r="nm"||ln="list"
m="ranitidine hcl syrup" 19:0 19:2||do="150 mg" 19:3 19:4||mo="po" 19:5 19:5||f="bid" 19:6 19:6||du="nm"||r="nm"||ln="list"
m="roxicet elixir ( oxycodone+apap liquid )" 20:0 20:5||do="5-10 milliliters" 21:0 21:1||mo="po" 21:2 21:2||f="q4h prn" 21:3 21:4||du="nm"||r="pain" 21:5 21:5||ln="list"
m="colace ( docusate sodium )" 22:0 22:4||do="100 mg" 22:5 22:6||mo="po" 22:7 22:7||f="tid" 22:8 22:8||du="nm"||r="nm"||ln="list"
m="phenergan ( promethazine hcl )" 24:0 24:4||do="25 mg" 24:5 24:6||mo="pr" 24:7 24:7||f="q6h prn" 24:8 24:9||du="nm"||r="nausea" 24:10 24:10||ln="list"
m="augmentin susp. 250mg/62.5 mg ( 5ml ) ( amoxicil... )" 25:0 25:9||do="10 milliliters" 26:0 26:1||mo="po" 26:2 26:2||f="tid" 26:3 26:3||du="for five days" 26:5 26:7||r="nm"||ln="list"
m="pca analgesia" 51:11 51:12||do="nm"||mo="nm"||f="nm"||du="nm"||r="pain" 51:6 51:6||ln="narrative"
m="e

In [30]:
n = 3
print([labelled_cases.data[n].test_text[a] for a in range(len(labelled_cases.data[n].test_text)) if labelled_cases.data[n].test_labels[a] == [1, 0]])

[]


In [ ]:
train_set = [model[word] if word in model.wv.vocab else np.zeros(100) for case in labelled_cases.data[:230] for word in case.test_text]
train_labels = [label for case in labelled_cases.data[:230] for label in case.test_labels] 
test_set = [model[word] if word in model.wv.vocab else np.zeros(100) for case in labelled_cases.data[230:] for word in case.test_text]
test_labels = [label for case in labelled_cases.data[230:] for label in case.test_labels]

test_words = [word for case in labelled_cases.data[230:] for word in case.test_text]
print(*[len(a) for a in [train_set, train_labels, test_set, test_labels, test_words]])

In [ ]:
test_words[-1000:]

In [ ]:
print(*map(len, [train_set, train_labels, test_set, test_labels, test_words]))

In [ ]:
print(np.array(train_labels).sum(0)/ len(train_labels))
print(len(train_set))

In [ ]:
while (np.array(train_labels).sum(0)/ len(train_labels))[0] < 0.5:
    for med in target_dict['medications']:
        train_set.append(model_I2B2[med])
        train_labels.append([1,0])
    print(np.array(train_labels).sum(0)/len(train_labels))

In [ ]:
print(np.array(train_labels).sum(0)/ len(train_labels))

In [ ]:
node_count = 10
input_size = 100
output_size = 2
batch_size = 50
epochs = 10
report_percentage = 10

In [ ]:
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.05)
        return tf.Variable(initial)

    def bias_variable(shape):
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)

    node_count = node_count

    x = tf.placeholder(tf.float32, shape=[None, input_size])
    y_ = tf.placeholder(tf.float32, shape=[None, output_size])

    # Define the first layer here
    W = weight_variable([input_size, node_count])
    b = bias_variable([node_count])
    h = tf.nn.sigmoid(tf.matmul(x, W) + b)

    # Use dropout for this layer (should you wish)
    # keep_prob = tf.placeholder(tf.float32)
    # h_drop = tf.nn.dropout(h1, keep_prob)

    # Define the second layer here
    # W2 = weight_variable([node_count_1, node_count_2])
    # b2 = bias_variable([node_count_2])
    # h2 = tf.nn.sigmoid(tf.matmul(h, W) + b)

    # Define the output layer here
    V = weight_variable([node_count, output_size])
    c = bias_variable([output_size])
    y = tf.nn.softmax(tf.matmul(h, V) + c)

    # We'll use the cross entropy loss function
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))

    # And classification accuracy
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # And the Adam optimiser
    train_step = tf.train.AdamOptimizer(learning_rate=1e-2).minimize(cross_entropy)

    # Start a tf session and run the optimisation algorithm
    sess = tf.Session()
    # sess.run(tf.initialize_all_variables())
    sess.run(tf.global_variables_initializer())

    training = Iterator(train_set, train_labels)
    mark = (epochs * (len(train_set) // batch_size) * report_percentage) // 100
    N = 0

    #print("Target: %s \tRepetitions: %s" % (target, repetitions))
    while training.epochs < epochs:
        trd, trl = training.next_batch(batch_size)
        if N % mark == 0:
            train_accuracy = sess.run(accuracy, feed_dict={x: trd, y_: trl})
            test_accuracy = sess.run(accuracy, feed_dict={x: test_set, y_: test_labels})
            print("Progress: %d%% \tTraining Accuracy: %f\t Test accuracy: %f" % (N * report_percentage// mark, train_accuracy, test_accuracy))
        sess.run(train_step, feed_dict={x: trd, y_: trl})
        N += 1

    print("Final Test Accuracy: %f\n" % (sess.run(accuracy, feed_dict={x: test_set, y_: test_labels})))

In [ ]:
pred = tf.argmax(y, 1)
res = sess.run(pred, feed_dict={x: test_set, y_:test_labels})
tru = np.argmax(test_labels, 1)

In [ ]:
sk.metrics.f1_score(tru, res, pos_label=0, average='binary')

In [ ]:
test_words = [word for word in case.test_text for case in labelled_cases.data[230:]]

In [ ]:
[(test_words[a], res[a], tru[a]) for a in [i*28] if res[a] == 1 and tru[a] == 1]

In [ ]:
print(test_words)

In [ ]:
len([a for a in res if a == 0])

In [ ]:
sk.metrics.f1_score(tru, res, pos_label=0, average='binary')

In [ ]:
[(example_text_5[a], res[a], tru[a]) for a in range(len(example_text_5)) if res[a] == 0 and tru[a] == 1]

In [ ]:
tp = len([a for a in range(len(tru)) if (res[a] == 0) and (tru[a] == 0)])
tn = len([a for a in range(len(tru)) if (res[a] == 1) and (tru[a] == 1)])
fp = len([a for a in range(len(tru)) if (res[a] == 0) and (tru[a] == 1)])
fn = len([a for a in range(len(tru)) if (res[a] == 1) and (tru[a] == 0)])
tp, tn, fp, fn

In [ ]:
print((2*tp)/(2*tp + fn +fp))